<a href="https://colab.research.google.com/github/athanoid/pycovid19/blob/master/covid19_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
A Polynomial Regression model with COVID-19 datasets

Data Repository by Johns Hopkins CSSE: https://github.com/CSSEGISandData/COVID-19

@author: thanos
"""
import pandas as pd
import numpy as np

In [42]:
#import data

# fetch from web
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

# assign to dataframe
df = confirmed
#show 5 first rows
df.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,14,14,14,19,19,19,19,25,25,25,25,32,32,32,33,33,33,33,33,34,35,35,35,35,35,35,35,35,37,40,40,41,42,42,43,43,43,47,48,50,50,50,53,59,70,75,82,114,147,177,212,272,322
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,7,7,11,15,20,20,20,22,22,45,25,25,26,26,26,28,28,29,43,59,66,74,84,94,105,122,147,159,170,189,214,228,241,256,274,293,331,360,420,461,502,511,581,639,639,701,773,839,825,878,889,924,963
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,7,7,10,13,16,18,18,24,28,28,30,33,40,45,47,50,58,67,72,75,77,81,84,84,85,85,89,89,91,93,93,93,102,106,108,110,110,117,130,138,150,150,160,178,178,200,212,226,243,266,313,345,385
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,4,7,8,8,8,8,8,10,12,12,12,16,16,18,18,18,19,19,22,22,22,22,22,22,22,22,22,22,22,22,23,23,25,29,29,36,50,50,83,93,99,117,129,149,149,197,238,428,566,673,790,900,1030


In [43]:
# create datasets from DataFrame

# get available dates in mm/dd/yyyy format
dates = list(df)
dates = dates[4:np.size(dates)]

# select country of interest
country = 'Greece'
loc = df.loc[df['Country/Region'] == country]
idx = int(loc.index.values)
#get only numeric data
num = df._get_numeric_data()
# data preparation 
y = list(num.values[idx,2: int(num.size)]) #get list of values
y = np.array(y) #convert into a numpy array
y = y[~np.isnan(y)] #remove nan entries
x = np.array(range(np.size(y)))#to numpy array
x = x[~np.isnan(x)] #remove nan entries
X = x.reshape(-1, 1) #requires reshaping for poly_reg.fit 

# if nans are removed, check if dates list has the same size as y and equalize
if np.abs(np.size(dates)-np.size(y)) > 0:
    diff = np.size(dates)-np.size(y)
    del dates[-diff]
    print('\nRemoved ' + str(diff) + ' value(s)\n')

print('\nLast 10 points of arrays:')
print('x: ' + str(dates[np.size(dates)-10: np.size(dates)]))
print('y: ' + str(y[np.size(y)-10: np.size(y)]))



Last 10 points of arrays:
x: ['3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20']
y: [ 99.  99. 190. 228. 331. 331. 387. 418. 418. 495.]


In [44]:
# stacked bar chart
import plotly.graph_objects as go

cnf = confirmed.loc[df['Country/Region'] == country]._get_numeric_data().values[0,np.size(dates)+1]
dth = deaths.loc[df['Country/Region'] == country]._get_numeric_data().values[0,np.size(dates)+1]
rcv = recovered.loc[df['Country/Region'] == country]._get_numeric_data().values[0,np.size(dates)+1]

xxs=[country]
fig = go.Figure(go.Bar(x=xxs, y=[cnf], name='Confirmed'))
fig.add_trace(go.Bar(x=xxs, y=[dth], name='Deaths'))
fig.add_trace(go.Bar(x=xxs, y=[rcv], name='Recovered'))

fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['Confirmed', 'Deaths', 'Recovered']})
fig.show()

In [45]:
# Fitting a Polynomial Regression to the dataset
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

poly_reg = PolynomialFeatures(degree=4)
X_poly = poly_reg.fit_transform(X)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, y)

y_pred = pol_reg.predict(poly_reg.fit_transform(X))#predicted values

rsqrd = r2_score(y, y_pred) #r-squared
print('r-squared: '+str(rsqrd))

rmse = np.linalg.norm(y_pred - y) / np.sqrt(len(y_pred)) #Root mean squared error 
print('rmse: '+str(rmse))

r-squared: 0.982127370711691
rmse: 16.508351874248614


In [46]:
#plot time series with range-slider
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=np.array(dates),
                y=y,
                name="Confirmed",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=np.array(dates),
                y=y_pred,
                name="Predicted",
                line_color='dimgray',
                opacity=0.8))

fig.update_layout(title_text="Country: "+country,
                  xaxis_rangeslider_visible=True)

fig.show()

In [47]:
#plot forecasted values
days = 14 #days ahead
x_plus = np.arange(np.size(x),np.size(y)+ days) 
x_plus = np.append([x], [x_plus])#append days to forecast
xf = x_plus.reshape(-1, 1)
yf = pol_reg.predict(poly_reg.fit_transform(xf))

# Visualize the Polymonial Regression results
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=x_plus,
                y=y,
                name="Confirmed",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=x_plus,
                y=yf,
                name="Predicted with projection",
                line_color='dimgray',
                opacity=0.8))

fig.update_layout(title_text="Country: " + country + " ("+ str(days) +" days projection)",
                  xaxis_rangeslider_visible=True)

fig.show()

In [48]:
# print report
print('---\n'+ country + ' - latest data: ' + str(dates[np.size(dates)-1]))
print('\nCurrent cases: '+ str([y[np.size(y)-1]]) + '\nPredicted: ' + str(np.round(pol_reg.predict(poly_reg.fit_transform([[np.size(y)]])),1)) + '\nDiff: ' + str(np.round(y[np.size(y)-1] - pol_reg.predict(poly_reg.fit_transform([[np.size(y)]])),1) )+'\nr-squared: '+ str(np.round(rsqrd,2)))
print('\nNext day prediction: '+str(np.round(pol_reg.predict(poly_reg.fit_transform([[np.size(y) + 1]])))) )
#approximate doubled incidents
for i in range(0, int(xf[np.size(xf)-1])):
    if np.isclose(np.around(y[np.size(y)-1]*2), np.round(yf[i-1],0), atol=100):
        #print(i)
        dday = i
        break
double_days = dday - int(x[np.size(x)-1])
print('\nIncidents will double approximately in ' + str(double_days) + ' days' + '\n---')

---
Greece - latest data: 3/20/20

Current cases: [495.0]
Predicted: [585.3]
Diff: [-90.3]
r-squared: 0.98

Next day prediction: [658.]

Incidents will double approximately in 6 days
---
